# <u>T16 HOI HUB</u>

## Import Libraries

In [ ]:
from ipywidgets import widgets
from IPython.display import display
import os
import io
import base64

## Data Exploration

In [ ]:
video_folder = os.listdir('data/video')

list_of_video = {}
for video in video_folder:
    # remove hidden file especially for macOS
    if not video.startswith('.'):
        list_of_video[video] = video

In [ ]:
video_dropdown = widgets.Dropdown(
    options = list_of_video,
    description = 'Videos',
)

video_src = 'data/video/' + video_dropdown.value

def play_video(video_src):
    video = io.open(video_src, 'r+b').read()
    encoded = base64.b64encode(video)
    return(HTML(data='''<video width="650" height="360" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii'))))


# video dropdown onchange function
def video_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']
        global video_src 
        video_src = 'data/video/'+ video_dropdown.value
        
          
# display video dropdown
video_dropdown.observe(video_on_change)
display(video_dropdown) 


video = video_src.split('/')[-1]
print("Currently playing : " + video)
play_video(video_src)

## Inference

## Training HOI ML Model

## Evaluate